In [ ]:
import os
import re
import glob
import pickle
import json
import logging

from bidict import bidict

from ptm import AuthorTopicModel
from ptm.nltk_corpus import get_ids_cnt
from ptm.utils import convert_cnt_to_list

PTM_ROOT = "/Users/altaf/Projects/dnm_evolution/python-topic-model"

In [ ]:
FORMAT = '%(asctime)-15s %(levelname)-6s %(message)s'
DATE_FORMAT = '%b %d %H:%M:%S'
formatter = logging.Formatter(fmt=FORMAT, datefmt=DATE_FORMAT)
handler = logging.StreamHandler()
handler.setFormatter(formatter)
logger = logging.getLogger('AuthorTopicModel')

logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

In [ ]:
DATA_DIR = "/Users/altaf/Projects/dnm_evolution/data/agora/posts"
AUTHORS_FILENAME = os.path.join(DATA_DIR, "authors.json")

def slurp_file(filename):
    with open(filename, 'r') as f:
        return(f.read().replace('\n', ' '))

files = glob.glob(DATA_DIR + "/*.txt")

In [7]:
with open(AUTHORS_FILENAME, "r") as f:
    authors = json.load(f)

author_db = bidict()

def get_author_id(filename):
    match = re.match("post_(.*?)\.", os.path.basename(filename))
    post_id = match.group(1)
    if not post_id in authors:
        return -1
    author = authors[post_id]

    author_id = author_db.get(author, -1)
    if (author_id < 0):
        author_id = len(author_db)
        author_db[author] = author_id
    return(author_id)

author_ids = [get_author_id(f) for f in files]
#author_ids = filter(lambda x: x > 0, author_ids)

In [9]:
files = files[0:len(authors)]

print("slurping files")
posts = [slurp_file(f) for f in files]
#print("converting to corpus, stage 1")
#voca, doc_ids, doc_cnt = get_ids_cnt(posts)
#print("converting to corpus, stage 2")
#corpus = convert_cnt_to_list(doc_ids, doc_cnt)

slurping files


In [ ]:
NUM_TOPICS = 50
MAX_ITER = 50

model = AuthorTopicModel(len(corpus), len(voca), NUM_TOPICS, len(author_ids))
model.fit(corpus, author_ids, max_iter = MAX_ITER)

In [11]:
CORPUS_TSV = "/Users/altaf/Projects/dnm_evolution/data/agora/posts/corpus.tsv"

delimiters = ' |;|,|\*|\?|\.|\n'
with open(CORPUS_TSV, 'w') as f:
    for i in range(1, len(posts)):
        tokens = filter(lambda x: x, re.split(delimiters, posts[i].lower()))
        f.write("%s\t%s\n" % (author_db.inv[author_ids[i]], ":".join(tokens)))

In [10]:
len(posts)

311675